In [2]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to C:\Users\Josep
[nltk_data]     Alet\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [94]:
from collections import defaultdict
from array import array
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
import math
import numpy as np
import collections
from numpy import linalg as la
import time
import pandas as pd
import json

In [16]:
docs_path = 'data/tweet_document_ids_map.csv'
with open(docs_path) as fp:
    lines = fp.readlines()
lines = [l.strip().replace(' +', ' ') for l in lines]
print("Total number of tweet documents in the corpus: {}".format(len(lines)))

Total number of tweet documents in the corpus: 4000


In [119]:
tweets_path = 'data/tw_hurricane_data.json'
with open(tweets_path) as fp:
    tweets = fp.readlines()
tweets = [json.loads(t) for t in tweets]
print("Total number of tweets in the corpus: {}".format(len(tweets)))

JSONDecodeError: Expecting value: line 2 column 1 (char 1)

In [116]:
t = [json.loads(tweets[0]), json.loads(tweets[1])]
print(type(t[0]))
f = pd.DataFrame.from_dict(data=t)
print(f.head())

<class 'dict'>
                       created_at                   id               id_str  \
0  Fri Sep 30 18:39:08 +0000 2022  1575918182698979328  1575918182698979328   
1  Fri Sep 30 18:39:01 +0000 2022  1575918151862304768  1575918151862304768   

                                           full_text  truncated  \
0  So this will keep spinning over us until 7 pm…...      False   
1  Our hearts go out to all those affected by #Hu...      False   

  display_text_range                                           entities  \
0            [0, 76]  {'hashtags': [{'text': 'HurricaneIan', 'indice...   
1           [0, 136]  {'hashtags': [{'text': 'HurricaneIan', 'indice...   

                                   extended_entities  \
0  {'media': [{'id': 1575918178261254162, 'id_str...   
1                                                NaN   

                                            metadata  \
0  {'iso_language_code': 'en', 'result_type': 're...   
1  {'iso_language_code': 'en', 'result

In [103]:
tweets_dt = pd.DataFrame(tweets, columns=['created_at', 'id', 'id_str', 'full_text', 'truncated', 'display_text_range', 'entities', 'extended_entities', 'metadata', 'source', 'in_reply_to_status_id', 'in_reply_to_status_id_str', 'in_reply_to_user_id', 'in_reply_to_user_id_str', 'in_reply_to_screen_name', 'user', 'contributors', 'is_quote_status', 'retweet_count', 'favorite_count', 'favorited', 'retweeted', 'possibly_sensitive', 'lang'])
print(tweets_dt.head())

ValueError: Shape of passed values is (4001, 1), indices imply (4001, 24)

In [17]:
def build_terms(line):
    """
    Preprocess the article text (title + body) removing stop words, stemming,
    transforming in lowercase and return the tokens of the text.
    
    Argument:
    line -- string (text) to be preprocessed
    
    Returns:
    line - a list of tokens corresponding to the input text after the preprocessing
    """

    stemmer = PorterStemmer()
    stop_words = set(stopwords.words("english"))
    ## START CODE
    line= line.lower() ## Transform in lowercase
    line= line.split() ## Tokenize the text to get a list of terms
    line=[x for x in line if x not in stop_words]  ##eliminate the stopwords (HINT: use List Comprehension)
    line=[stemmer.stem(x) for x in line] ## perform stemming (HINT: use List Comprehension)
    ## END CODE
    return line

In [89]:
for line in lines:
    print(build_terms(line))

['doc_1', '1575918182698979328']
['doc_2', '1575918151862304768']
['doc_3', '1575918140839673873']
['doc_4', '1575918135009738752']
['doc_5', '1575918119251419136']
['doc_6', '1575918105854984192']
['doc_7', '1575918095008681986']
['doc_8', '1575918088473788429']
['doc_9', '1575918083075555329']
['doc_10', '1575918057037303808']
['doc_11', '1575918052595552256']
['doc_12', '1575918038196666392']
['doc_13', '1575918028717707265']
['doc_14', '1575918018991099904']
['doc_15', '1575918018235813888']
['doc_16', '1575918012221333504']
['doc_17', '1575918004642402304']
['doc_18', '1575917983062380545']
['doc_19', '1575917982726860802']
['doc_20', '1575917943426535424']
['doc_21', '1575917938502254602']
['doc_22', '1575917915882373137']
['doc_23', '1575917865743663125']
['doc_24', '1575917833573179392']
['doc_25', '1575917821560864782']
['doc_26', '1575917776614871040']
['doc_27', '1575917773376540672']
['doc_28', '1575917720499109910']
['doc_29', '1575917719416979466']
['doc_30', '15759177176

In [18]:
print(build_terms("One day I saw a cow without a tie dressed with a uniform"))

['one', 'day', 'saw', 'cow', 'without', 'tie', 'dress', 'uniform']


In [81]:
def create_index(lines):
    """
    Implement the inverted index
    
    Argument:
    lines -- collection of Wikipedia articles
    
    Returns:
    index - the inverted index (implemented through a Python dictionary) containing terms as keys and the corresponding
    list of documents where these keys appears in (and the positions) as values.
    """
    index = defaultdict(list)
    title_index = {}  # dictionary to map page titles to page ids
    for line in lines:  # Remember, lines contain all documents: article-id | article-title | article-body
        #line_arr = line.split("\")
        #print(line.split("\t"))
        line_arr=line.split("\t")[1]
        page_id = int(line_arr)
        terms = build_terms(''.join(line_arr[1:])) # page_title + page_text
        title = line_arr[1]
        title_index[page_id]=title  ## we do not need to apply get terms to title because it used only to print titles and not in the index
        
        ## ===============================================================        
        ## create the index for the current page and store it in current_page_index (current_page_index)
        ## current_page_index ==> { ‘term1’: [current_doc, [list of positions]], ...,‘term_n’: [current_doc, [list of positions]]}

        ## Example: if the curr_doc has id 1 and his text is 
        ##"web retrieval information retrieval":

        ## current_page_index ==> { ‘web’: [1, [0]], ‘retrieval’: [1, [1,3]], ‘information’: [1, [2]]}

        ## the term ‘web’ appears in document 1 in positions 0, 
        ## the term ‘retrieval’ appears in document 1 in positions 1 and 4
        ## ===============================================================

        current_page_index = {}

        for position, term in enumerate(terms): # terms contains page_title + page_text. Loop over all terms
            try:
                # if the term is already in the index for the current page (current_page_index)
                # append the position to the corresponding list
                
        ## START CODE
                current_page_index[term][1].append(position)  
            except:
                # Add the new term as dict key and initialize the array of positions and add the position
                current_page_index[term]=[page_id, array('I',[position])] #'I' indicates unsigned int (int in Python)
            
        #merge the current page index with the main index
        for term_page, posting_page in current_page_index.items():
            index[term_page].append(posting_page)
        
        ## END CODE                    
                    
    return index, title_index

In [78]:
s="d\th0"
print(s.split("\t"))

['d', 'h0']


In [83]:
start_time = time.time()
index, title_index = create_index(lines)
print("Total time to create the index: {} seconds".format(np.round(time.time() - start_time, 2)))

Total time to create the index: 3.61 seconds


In [87]:
index

defaultdict(list,
            {'575918182698979328': [[1575918182698979328, array('I', [0])]],
             '575918151862304768': [[1575918151862304768, array('I', [0])]],
             '575918140839673873': [[1575918140839673873, array('I', [0])]],
             '575918135009738752': [[1575918135009738752, array('I', [0])]],
             '575918119251419136': [[1575918119251419136, array('I', [0])]],
             '575918105854984192': [[1575918105854984192, array('I', [0])]],
             '575918095008681986': [[1575918095008681986, array('I', [0])]],
             '575918088473788429': [[1575918088473788429, array('I', [0])]],
             '575918083075555329': [[1575918083075555329, array('I', [0])]],
             '575918057037303808': [[1575918057037303808, array('I', [0])]],
             '575918052595552256': [[1575918052595552256, array('I', [0])]],
             '575918038196666392': [[1575918038196666392, array('I', [0])]],
             '575918028717707265': [[1575918028717707265, 